In [1]:
# Importamos las librerias que usaremos para el correcto funcionamiento del modelo LSTM y GARCH
import yfinance as yf
import numpy as np
import pandas as pd
from matplotlib import font_manager
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape
from sklearn.metrics import mean_absolute_error
from arch import arch_model
from itertools import groupby
from tensorflow.keras.losses import Huber
import tensorflow as tf
import math 
import warnings  
from matplotlib import font_manager
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
# Lista de símbolos de 100 Acciones de diferentes empresas del indice S&P 500
symbols = ['ABT', 'ADM', 'LRCX', 'AMAT', 'DHR', 'DE', 'DAL', 'ECL', 'CPB', 'BDX', 'BBY', 'BIIB', 'GILD', 'ACN', 'AMGN', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'UPS', 'ADSK', 'ADP', 'AXP', 'AMD', 'ADBE', 'MMM', 'VZ', 'BAC', 'BK', 'AVGO', 'BA', 'BMY', 'COF', 'CAT', 'CVX', 'CSCO', 'C', 'KO', 'CL', 'CMCSA', 'ABBV', 'CNC', 'SCHW', 'COP', 'CVS', 'NEE', 'DIS', 'MDT', 'DUK', 'ETN', 'DD', 'XOM', 'GE', 'FDX', 'F', 'GM', 'GS', 'HAL', 'HOG', 'HON', 'INTC', 'INTU', 'JPM', 'JNJ', 'JCI', 'KMB', 'TSLA', 'LNC', 'LMT', 'LOW', 'MA', 'MCD', 'MRK', 'ORCL', 'MDLZ', 'COST', 'MS', 'NFLX', 'NKE', 'PEP', 'PFE', 'PG', 'PGR', 'PRU', 'QCOM', 'RTX', 'RSG', 'CRM', 'SSNLF', 'SNY', 'SLB', 'ALL', 'FAST', 'FIS', 'EA', 'EIX', 'EMR', 'EOG', 'TGT']

# Verificamos si existen símbolos repetidos en el código anterior
repeated_symbols = set()
unique_symbols = set()

for symbol in symbols:
    if symbol in unique_symbols:
        repeated_symbols.add(symbol)
    else:
        unique_symbols.add(symbol)

if repeated_symbols:
    print("Estos son los símbolos repetidos en la lista:")
    for symbol in repeated_symbols:
        print(symbol)
else:
    print("No hay símbolos repetidos en la lista analizada.")

No hay símbolos repetidos en la lista analizada.


In [3]:
# Separamos los símbolos por la primera letra de abecedario para dar orden
symbols_by_first_letter = {k: list(g) for k, g in groupby(sorted(symbols), key=lambda x: x[0])}

# Imprimimos los símbolos en columnas por la primera letra del abecedario
print("Acciones únicas por letra del abecedario que no se repiten:")
for letter, symbols_list in symbols_by_first_letter.items():
    symbols_str = ', '.join(symbols_list)
    print(f"{letter}: {symbols_str}")

Acciones únicas por letra del abecedario que no se repiten:
A: AAPL, ABBV, ABT, ACN, ADBE, ADM, ADP, ADSK, ALL, AMAT, AMD, AMGN, AMZN, AVGO, AXP
B: BA, BAC, BBY, BDX, BIIB, BK, BMY
C: C, CAT, CL, CMCSA, CNC, COF, COP, COST, CPB, CRM, CSCO, CVS, CVX
D: DAL, DD, DE, DHR, DIS, DUK
E: EA, ECL, EIX, EMR, EOG, ETN
F: F, FAST, FDX, FIS
G: GE, GILD, GM, GOOGL, GS
H: HAL, HOG, HON
I: INTC, INTU
J: JCI, JNJ, JPM
K: KMB, KO
L: LMT, LNC, LOW, LRCX
M: MA, MCD, MDLZ, MDT, MMM, MRK, MS, MSFT
N: NEE, NFLX, NKE, NVDA
O: ORCL
P: PEP, PFE, PG, PGR, PRU
Q: QCOM
R: RSG, RTX
S: SCHW, SLB, SNY, SSNLF
T: TGT, TSLA
U: UPS
V: VZ
X: XOM


In [4]:
# Descargar datos históricos de las 100 acciones de empresas del indice S&P 500 desde el año 2015 hasta el año 2023
data_frames = []
download_count = 1

for symbol in symbols:
    # Descargar datos históricos de las 100 acciones del S&P 500
    stock_data = yf.download(symbol, start='2015-01-01', end='2023-12-31')

    # Añadimos a la lista solo las columnas que deseamos ver, precio cierre y volumen
    if ('Close' in stock_data.columns) and ('Volume' in stock_data.columns):
        data_frames.append(stock_data[['Close', 'Volume']])
        print(f"Descarga {download_count}: Datos históricos de {symbol} descargados.")
        download_count += 1
    else:
        print(f"No se encontraron las columnas 'Close' y 'Volume' para {symbol}.")

[*********************100%%**********************]  1 of 1 completed


Descarga 1: Datos históricos de ABT descargados.


[*********************100%%**********************]  1 of 1 completed


Descarga 2: Datos históricos de ADM descargados.


[*********************100%%**********************]  1 of 1 completed


...

Descarga 98: Datos históricos de EMR descargados.


[*********************100%%**********************]  1 of 1 completed


Descarga 99: Datos históricos de EOG descargados.


[*********************100%%**********************]  1 of 1 completed

Descarga 100: Datos históricos de TGT descargados.


In [5]:
# Lista de DataFrames que contienen las columnas 'Close' y 'Volume' de cada empresa seleccionada
# Concatenamos los datos
data = pd.concat(data_frames, axis=1, keys=symbols, names=['Symbol', 'Data'])

# Reemplazamos posibles datos infinitos por NaN
data = data.replace([np.inf, -np.inf], np.nan)

# Filtrar las filas con valores NaN en alguna columna
filas_con_nan = data[data.isna().any(axis=1)]
print(filas_con_nan)

# Encontrar las columnas con valores NaN
columnas_con_nan = data.columns[data.isna().any()].tolist()
print("Columnas con valores NaN:", columnas_con_nan)

Empty DataFrame
Columns: [(ABT, Close), (ABT, Volume), (ADM, Close), (ADM, Volume), (LRCX, Close), (LRCX, Volume), (AMAT, Close), (AMAT, Volume), (DHR, Close), (DHR, Volume), (DE, Close), (DE, Volume), (DAL, Close), (DAL, Volume), (ECL, Close), (ECL, Volume), (CPB, Close), (CPB, Volume), (BDX, Close), (BDX, Volume), (BBY, Close), (BBY, Volume), (BIIB, Close), (BIIB, Volume), (GILD, Close), (GILD, Volume), (ACN, Close), (ACN, Volume), (AMGN, Close), (AMGN, Volume), (AAPL, Close), (AAPL, Volume), (MSFT, Close), (MSFT, Volume), (GOOGL, Close), (GOOGL, Volume), (AMZN, Close), (AMZN, Volume), (NVDA, Close), (NVDA, Volume), (UPS, Close), (UPS, Volume), (ADSK, Close), (ADSK, Volume), (ADP, Close), (ADP, Volume), (AXP, Close), (AXP, Volume), (AMD, Close), (AMD, Volume), (ADBE, Close), (ADBE, Volume), (MMM, Close), (MMM, Volume), (VZ, Close), (VZ, Volume), (BAC, Close), (BAC, Volume), (BK, Close), (BK, Volume), (AVGO, Close), (AVGO, Volume), (BA, Close), (BA, Volume), (BMY, Close), (BMY, Volume

In [6]:
# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Ajustamos el índice y las columnas
data = pd.DataFrame(scaled_data, index=data.index, columns=data.columns)

# Parámetros elegidos para el modelo LSTM
SEQ_LEN = 5  # Días de historia
PRED_LEN = 2  # Días a predecir en el futuro

In [7]:
# Función para preprocesar los datos de la serie temporal para prepararlos para su uso en un modelo de red LSTM
def preprocess_data(data, seq_len, pred_len):
    X, y = [], []

    for i in range(len(data) - seq_len - pred_len):
        # Secuencia de entrada
        seq = data[i:i + seq_len]
        x = seq.values.reshape(1, seq_len, data.shape[1])
        X.append(x)

        # Predicción a futuro
        pred = data.iloc[i + seq_len:i + seq_len + pred_len].values
        y.append(pred)

    return np.array(X), np.array(y)

In [8]:
# Preprocesar los datos
X, y = preprocess_data(pd.DataFrame(scaled_data), SEQ_LEN, PRED_LEN)
X = np.squeeze(X, axis=1)

# Verificar la presencia de NaN o infinitos en los conjuntos de entrenamiento y prueba
print(np.isnan(X).any(), np.isnan(y).any())
print(np.isinf(X).any(), np.isinf(y).any())

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)


False False
False False


In [9]:
# Fijamos una semilla en Numpy 
seed = 5
np.random.seed(seed)
tf.random.set_seed(seed)

# Generar números aleatorios con numpy
np_random_numbers = np.random.rand(5)
print("Números aleatorios de numpy:", np_random_numbers)

# Generar números aleatorios con TensorFlow
tf_random_numbers = tf.random.uniform([5])
print("Números aleatorios de TensorFlow:", tf_random_numbers)

Números aleatorios de numpy: [0.22199317 0.87073231 0.20671916 0.91861091 0.48841119]
Números aleatorios de TensorFlow: tf.Tensor([0.6263931  0.5298432  0.7584572  0.5084884  0.34415376], shape=(5,), dtype=float32)


In [10]:
# Configuración del modelo LSTM
model = Sequential()
model.add(LSTM(units=500, activation='relu', input_shape=(SEQ_LEN, data.shape[1])))
model.add(Dense(PRED_LEN * data.shape[1], activation='linear'))
model.add(Reshape((PRED_LEN, data.shape[1])))
model.compile(optimizer='adam', loss=Huber())

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=200, batch_size=32)

# Evaluación del modelo
loss = model.evaluate(X_test, y_test)
print(f'Loss en conjunto de prueba: {loss}')

# Realizar predicciones
predictions = model.predict(X_test)



Epoch 1/200

57/57 [==============================] - 3s 19ms/step - loss: 0.0090
Epoch 2/200
57/57 [==============================] - 1s 20ms/step - loss: 0.0025
Epoch 3/200
57/57 [==============================] - 1s 20ms/step - loss: 0.0020
Epoch 4/200
57/57 [==============================] - 1s 22ms/step - loss: 0.0018
Epoch 5/200
57/57 [==============================] - 1s 20ms/step - loss: 0.0017
Epoch 6/200
57/57 [==============================] - 1s 22ms/step - loss: 0.0016
Epoch 7/200
57/57 [==============================] - 1s 24ms/step - loss: 0.0015
Epoch 8/200
57/57 [==============================] - 1s 26ms/step - loss: 0.0015
Epoch 9/200
57/57 [==============================] - 2s 27ms/step - loss: 0.0014
Epoch 10/200
57/57 [==============================] - 1s 26ms/step - loss: 0.0014
Epoch 11/200
57/57 [==============================] - 1s 19ms/step - loss: 0.0014
Epoch 12/200
57/57 [==============================] - 1s 17ms/step - loss: 0.0014
Epoch 13/200
57/57 [==

In [ ]:
# Función para aplicar suavizado exponencial
def exponential_smoothing(data, alpha):
    # Convertir a DataFrame de Pandas
    data_df = pd.DataFrame(data)
    smoothed_data = data_df.ewm(alpha=alpha, adjust=False).mean()
    return smoothed_data.values.flatten()  # Convertir de nuevo a array NumPy después del suavizado

# Definir el número de filas y columnas para los subplots
num_cols = 2  # número de columnas
num_rows = 3  # número de filas
plots_per_figure = num_cols * num_rows

# Colores para las predicciones
prediction_colors = ['blue', 'green', 'red', 'purple', 'brown'] * ((len(symbols) // 5) + 1)

# Tamaños de fuente y marcadores
font_size_title = 11
font_size_labels = 11
font_size_legend = 11
marker_size = 0.5

# Graficar predicciones y valores reales para cada empresa
for fig_num in range((len(symbols) // plots_per_figure) + 1):
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 15))
    axs = axs.flatten()

    for i in range(plots_per_figure):
        symbol_index = fig_num * plots_per_figure + i
        if symbol_index >= len(symbols):
            break
        symbol = symbols[symbol_index]
        ax = axs[i]

        # Aplicar suavizado exponencial a las predicciones
        smoothed_predictions = exponential_smoothing(predictions[:, :, symbol_index].reshape(-1), alpha=0.2)

        # Graficar predicciones suavizadas con colores diferentes
        ax.plot(smoothed_predictions, label=f'Predicciones {symbol} (Suavizadas)', color=prediction_colors[symbol_index], marker='o', linestyle='--', markersize=marker_size)

        # Graficar valores reales con color constante (naranja)
        ax.plot(y_test[:, :, symbol_index].reshape(-1), label=f'Valores Reales {symbol}', color='orange', marker='x', linestyle='-', markersize=marker_size)

        # Configurar título, etiquetas y leyenda
        ax.set_title(f'Predicciones vs Valores Reales para {symbol}', fontsize=font_size_title)
        ax.set_xlabel('Tiempo', fontsize=font_size_labels)
        ax.set_ylabel('Valor', fontsize=font_size_labels)
        ax.legend(fontsize=font_size_legend)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Calcular el MAE
mae_test = mean_absolute_error(y_test.reshape(-1, data.shape[1]), predictions.reshape(-1, data.shape[1]), multioutput='raw_values')

# Calcular el MSE
mse_test = mean_squared_error(y_test.reshape(-1, data.shape[1]), predictions.reshape(-1, data.shape[1]), multioutput='raw_values')

# Calcular el RMSE
rmse_test = np.sqrt(mse_test)

# Crear un DataFrame para organizar los resultados
df_results = pd.DataFrame({
    'Variable': range(1, data.shape[1] + 1),
    'MAE': mae_test,
    'MSE': mse_test,
    'RMSE': rmse_test
})

# Configurar Pandas para mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Imprimir los resultados ordenados
print(df_results)

In [ ]:
# Lista de símbolos de 100 Acciones de diferentes empresas del S&P 500
symbols = ['ABT', 'ADM', 'LRCX', 'AMAT', 'DHR', 'DE', 'DE', 'ECL', 'CPB', 'BDX', 'BBY', 'BIIB', 'GILD', 'ACN', 'AMGN', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'UPS', 'ADSK', 'ADP', 'AXP', 'AMD', 'ADBE', 'MMM', 'VZ', 'AMD', 'BK', 'AVGO', 'BA', 'BMY', 'COF', 'CAT', 'CVX', 'CSCO', 'C', 'KO', 'CL', 'CMCSA', 'ABBV', 'CNC', 'SCHW', 'COP', 'CVS', 'NEE', 'DIS', 'MDT', 'DUK', 'ETN', 'DD', 'XOM', 'GE', 'FDX', 'F', 'GM', 'GS', 'HAL', 'HOG', 'HON', 'INTC', 'INTU', 'JPM', 'JNJ', 'JCI', 'KMB', 'TSLA', 'LNC', 'LMT', 'LOW', 'MA', 'MCD', 'MRK', 'ORCL', 'MDLZ', 'COST', 'MS', 'NFLX', 'NKE', 'PEP', 'PFE', 'PG', 'PGR', 'PRU', 'QCOM', 'RTX', 'RSG', 'CRM', 'SSNLF', 'SNY', 'SLB', 'ALL', 'FAST', 'FIS', 'EA', 'EIX', 'EMR', 'EOG', 'TGT']

# Función para obtener datos históricos de una acción y calcular rentabilidad y volatilidad
def calcular_rentabilidad_y_volatilidad(symbol):
    # Obtener datos históricos desde el año 2015 hasta 2023
    data = yf.download(symbol, start="2015-01-01", end="2023-12-31")

    # Calcular la rentabilidad
    data['Rentabilidad'] = (data['Close'] / data['Close'].shift(1) - 1) * 100

    # Calcular la volatilidad
    data['Volatilidad'] = data['Rentabilidad'].rolling(window=252).std() * (252 ** 0.5)

    # Agrupar por año y calcular la rentabilidad media y la volatilidad media
    rentabilidad_anual = data['Rentabilidad'].resample('Y').mean()
    volatilidad_anual = data['Volatilidad'].resample('Y').mean()

    # Retornar los resultados por año
    return rentabilidad_anual, volatilidad_anual


In [ ]:
# Crear un DataFrame vacío para almacenar los resultados
resultados = pd.DataFrame(columns=['Símbolo', 'Año', 'Rentabilidad', 'Volatilidad'])

# Calcular rentabilidad y volatilidad para cada símbolo
for symbol in symbols:
    rentabilidad_anual, volatilidad_anual = calcular_rentabilidad_y_volatilidad(symbol)
    for year in rentabilidad_anual.index:
        resultados.loc[len(resultados)] = [symbol, year.year, rentabilidad_anual[year], volatilidad_anual[year]]

# Imprimir los resultados
print(resultados)

In [ ]:
# Convertir NaN a 0 en las columnas 'Rentabilidad' y 'Volatilidad'
resultados['Rentabilidad'] = resultados['Rentabilidad'].fillna(0)
resultados['Volatilidad'] = resultados['Volatilidad'].fillna(0)

In [ ]:
# Ordenar los símbolos alfabéticamente
symbols_ordenados = sorted(resultados['Símbolo'].unique())

# Lista para almacenar los símbolos con error
simbolos_con_error = []

# Contadores para rentabilidad y volatilidad
rentabilidad_graficos = 0
volatilidad_graficos = 0

# Definir el número de filas y columnas para los subplots
num_cols = 2  # número de columnas
num_rows = 3  # número de filas
plots_per_figure = num_cols * num_rows

In [ ]:
# Función para graficar un conjunto de subplots
def plot_subplots(symbols, tipo):
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 15))
    axs = axs.flatten()
    
    for i, symbol in enumerate(symbols):
        if i >= len(axs):
            break
        ax = axs[i]
        try:
            # Obtener los datos de la empresa actual
            grupo = resultados[resultados['Símbolo'] == symbol]
            
            if tipo == 'Rentabilidad':
                # Graficar la rentabilidad por año (gráfico de líneas)
                rentabilidad_por_ano = grupo.pivot(index='Año', columns='Símbolo', values='Rentabilidad')[symbol]
                rentabilidad_por_ano.plot(ax=ax, kind='line', color='orange', label='Rentabilidad')
                
                # Configurar etiquetas y título del gráfico de rentabilidad
                ax.set_title(f'Rentabilidad por Año - {symbol}')
                ax.set_xlabel('Año')
                ax.set_ylabel('Rentabilidad')
                ax.legend(loc='upper left')
                
                # Incrementar el contador de gráficos de rentabilidad
                global rentabilidad_graficos
                rentabilidad_graficos += 1
                
            elif tipo == 'Volatilidad':
                # Graficar la volatilidad por año (gráfico de líneas)
                volatilidad_por_ano = grupo.pivot(index='Año', columns='Símbolo', values='Volatilidad')[symbol]
                volatilidad_por_ano.plot(ax=ax, kind='line', color='blue', label='Volatilidad')
                
                # Configurar etiquetas y título del gráfico de volatilidad
                ax.set_title(f'Volatilidad por Año - {symbol}')
                ax.set_xlabel('Año')
                ax.set_ylabel('Volatilidad')
                ax.legend(loc='upper left')
                
                # Incrementar el contador de gráficos de volatilidad
                global volatilidad_graficos
                volatilidad_graficos += 1
                
        except Exception as e:
            # Agregar el símbolo a la lista de errores
            simbolos_con_error.append(symbol)
            print(f"Error al graficar {symbol}: {e}")
    
    # Ajustar el espaciado alrededor de los gráficos
    plt.tight_layout()
    
    # Mostrar los gráficos
    plt.show()

# Graficar rentabilidad
for fig_num in range((len(symbols_ordenados) // plots_per_figure) + 1):
    plot_subplots(symbols_ordenados[fig_num * plots_per_figure:(fig_num + 1) * plots_per_figure], 'Rentabilidad')

# Graficar volatilidad
for fig_num in range((len(symbols_ordenados) // plots_per_figure) + 1):
    plot_subplots(symbols_ordenados[fig_num * plots_per_figure:(fig_num + 1) * plots_per_figure], 'Volatilidad')

# Imprimir los símbolos que arrojaron error
if simbolos_con_error:
    print("Símbolos con error al graficar:")
    for simbolo in simbolos_con_error:
        print(simbolo)

# Imprimir el número de gráficos generados de rentabilidad y volatilidad
print(f"Número de gráficos de Rentabilidad: {rentabilidad_graficos}")
print(f"Número de gráficos de Volatilidad: {volatilidad_graficos}")

In [ ]:
# Lista de símbolos de 100 Acciones de diferentes empresas del S&P 500
symbols = ['ABT', 'ADM', 'LRCX', 'AMAT', 'DHR', 'DE', 'DE', 'ECL', 'CPB', 'BDX', 'BBY', 'BIIB', 'GILD', 'ACN', 'AMGN',
           'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'UPS', 'ADSK', 'ADP', 'AXP', 'AMD', 'ADBE', 'MMM', 'VZ', 'BAC', 'BK',
           'AVGO', 'BA', 'BMY', 'COF', 'CAT', 'CVX', 'CSCO', 'C', 'KO', 'CL', 'CMCSA', 'ABBV', 'CNC', 'SCHW', 'COP', 'CVS',
           'NEE', 'DIS', 'MDT', 'DUK', 'ETN', 'DD', 'XOM', 'GE', 'FDX', 'F', 'GM', 'GS', 'HAL', 'HOG', 'HON', 'INTC', 'INTU',
           'JPM', 'JNJ', 'JNJ', 'KMB', 'TSLA', 'LNC', 'LMT', 'LOW', 'MA', 'MCD', 'MRK', 'ORCL', 'MDLZ', 'COST', 'MS', 'NFLX',
           'NKE', 'PEP', 'PFE', 'PG', 'PGR', 'PRU', 'QCOM', 'RTX', 'RSG', 'CRM', 'SSNLF', 'SNY', 'SLB', 'ALL', 'FAST', 'FIS',
           'EA', 'EIX', 'EMR', 'EOG', 'TGT']

# DataFrame para almacenar las métricas de evaluación
metrics_df = pd.DataFrame(columns=['Symbol', 'AIC', 'BIC', 'Log-Likelihood', 'RMSE', 'MSE', 'MAE'])

# Descargar datos históricos para todos los símbolos a la vez
data = yf.download(symbols, start='2015-01-01', end='2023-01-01')

# Definir el número de filas y columnas para los subplots
num_cols = 2
num_rows = 3

# Inicializar el índice de la tabla
table_index = 1

for i, symbol in enumerate(symbols):
    # Preparación de los datos
    symbol_data = data['Adj Close'][symbol].dropna()
    returns = symbol_data.pct_change().dropna() * 100  # Convertir a porcentajes

    # División de los datos en conjuntos de entrenamiento (80%) y prueba (20%)
    train_size = int(0.8 * len(returns))
    train, test = returns[:train_size], returns[train_size:]

    # Ajuste del modelo GARCH(1,1) al conjunto de entrenamiento
    garch_model = arch_model(train, vol='Garch', p=1, q=1)
    garch_fit = garch_model.fit(disp="off")

    # Evaluación del modelo GARCH en el conjunto de entrenamiento
    aic_garch = garch_fit.aic
    bic_garch = garch_fit.bic
    log_likelihood_garch = garch_fit.loglikelihood

    # Predicciones paso a paso
    predictions = []
    for j in range(len(test)):
        model = arch_model(np.concatenate([train, test[:j]]), vol='Garch', p=1, q=1)
        model_fit = model.fit(disp="off")
        forecast = model_fit.forecast(horizon=1, reindex=False)
        predictions.append(forecast.variance.values[-1, 0])

    predicted_volatility = np.sqrt(predictions)

    # Cálculo de métricas en el conjunto de prueba
    rmse_garch = np.sqrt(mean_squared_error(test, predicted_volatility))
    mse_garch = mean_squared_error(test, predicted_volatility)
    mae_garch = mean_absolute_error(test, predicted_volatility)

    # Crear un DataFrame temporal para la fila actual
    temp_df = pd.DataFrame([{
        'Symbol': symbol,
        'AIC': aic_garch,
        'BIC': bic_garch,
        'Log-Likelihood': log_likelihood_garch,
        'RMSE': rmse_garch,
        'MSE': mse_garch,
        'MAE': mae_garch
    }])

    # Verificar si temp_df no está vacío o tiene todas las entradas NA antes de concatenar
    if not temp_df.empty and not temp_df.isna().all().all():
        metrics_df = pd.concat([metrics_df, temp_df], ignore_index=True)

    # Graficar los retornos y la volatilidad estimada por el modelo GARCH
    if i % (num_cols * num_rows) == 0:
        if i > 0:
            plt.tight_layout()
            plt.show()
        fig, ax = plt.subplots(num_rows, num_cols, figsize=(15, 10))
        ax = ax.flatten()

    ax[i % (num_cols * num_rows)].plot(returns, label='Returns')
    ax[i % (num_cols * num_rows)].plot(garch_fit.conditional_volatility, label='GARCH Volatility (Train)', color='blue')
    ax[i % (num_cols * num_rows)].plot(test.index, predicted_volatility, label='Predicted Volatility (Test)', color='red')
    ax[i % (num_cols * num_rows)].set_title(f'{symbol} GARCH(1,1) Conditional Volatility')
    ax[i % (num_cols * num_rows)].legend()

    # Imprimir métricas cada 10 símbolos
    if (i + 1) % 10 == 0:
        print(f"Tabla {table_index} - Métricas de Evaluación")
        print(metrics_df.iloc[-10:].to_string(index=False))
        table_index += 1

plt.tight_layout()
plt.show()

# Imprimir las métricas restantes si hay menos de 10 símbolos al final
if len(symbols) % 10 != 0:
    print(f"Tabla {table_index} - Métricas de Evaluación")
    print(metrics_df.iloc[-(len(symbols) % 10):].to_string(index=False))